In [1]:
#Run this cell to get the output
import requests
import json
import csv
from bs4 import BeautifulSoup
import traceback
import cloudscraper
import pandas as pd
import numpy as np
import seaborn as sns
from decimal import Decimal
from re import sub
import matplotlib.pyplot as plt
from datetime import date
import os
import shutil

postcode_buy="MK36JS" 
radius_buy=10

postcode_sell="DT13GJ" 
radius_sell=25

#Remeber to check the maximum and minimum years allowed on the website 
min_year=2008
max_year=2015

min_price=500
max_price=2000

#State how many brands/ models you want to show in the final graphs 
no_brands = 50
no_models = 50

import argparse

In [ ]:
def postcode_add(df,postcode):
    #Loop over the column names and add the postcode to each 
    for i in df.columns:
        concat_col = i + ' ' + postcode
        df = df.rename({i: concat_col.replace(' ','_')}, axis=1)

    return df

#function to retrieve cars based on parameters put in 
def retrieve_cars(postcode, radius, min_year, max_year, min_price, max_price,location_type):
    """
    Retrieve car listings based on specified search parameters.
    """
    cars = get_cars(
        postcode=postcode,
        radius=radius,
        min_year=min_year,
        max_year=max_year,
        min_price=min_price,
        max_price=max_price
    )
    # Clean up year and brand columns
    cars['year'] = cars['year'].dropna().apply(lambda x: int(str(x).split('(')[0]))
    cars['brand'] = cars['name'].apply(lambda x: x.split(' ')[0])
    cars['postcode'] = postcode
    cars = postcode_add(cars,location_type)
    
    return cars

In [ ]:
def get_cars(postcode,radius,min_year,min_year,max_year,min_price,max_price):

    #define pre url 
    url = "https://www.autotrader.co.uk/car-search?advertising-location=at_cars&exclude-delivery-option=on"

    keywords = {}
    keywords["mileage"] = ["miles"]
    keywords["year"] = [" reg)"]
    keywords["engine"] = ["engine"]

    params = {
        "sort": "relevance",
        "postcode": postcode,
        "price-from" : min_price,
        "price-to" : max_price,
        "radius": radius,
        "make": make,
        "model": model,
        "search-results-price-type": "total-price",
        "search-results-year": "select-year",
        "exclude-writeoff-categories": "on"
    }
    